![alt text](https://www.gameonaus.com/wp-content/uploads/2018/01/Dota2.jpg)

# Dota 2 Winning Team Predictor Engine
* * * 
*Dota 2 is a videogame of the MOBA genre with millions of active players and viewers. The videogame has become one of the most watched esports. Its biggest annual competition, "The international" will distribute a price pool in it's 10th edition of more than USD 40 millions to the winning team. In this project, I will build a engine for "VanN", a PRO Dota 2 player and streamer with 3 main functionalities: i. categorize characters into groups that help to understand team compositions and best strategies; ii. build a model that predicts the winning team before the game starts; and iii. build a model that predicts what team will win at an early stage of the game.*

## 1. Data
* * *
Data used in this project was obtained from OpenDota.com. This website provides information about millions of Dota 2 matches. Through its API service it is possible to download both macro and micro statistics and events of matches in a very granulated way. The data available can be split into "in-game" data (data gathered from teams or players throughout the game at certain timestamps) and "end-game" data (data gathered at end game, showing the accrued figures per team/player).

In this project, we will work with the [Kaggle dataset](https://www.kaggle.com/devinanzelmo/dota-2-matches) provided by Devin Anzelmo, which already includes several downloads from the OpenDota.com website of 50k Dota 2 games, including both "in-game" and "end-game" information of 500k players in 50k matches (10 players per match).

## 2. Method
* * *
There are 2 different tasks to accomplish. We will use a different methodology to cover each one of them:

1. **Grouping characters (heroes) into categories**: There is an extensive pool of heroes that players can choose to play with. In order to create groups of heroes, we will proceed to cluster them using the KNN algorithm due to it's simplicity. We will feed the KNN different variables of each heroe and group them into a certain number of clusters to map them into useful groups.

2. **Predicting which team will win the match**: We will use a supervised machine learning model. In particular, we will use the Random Forest Classifier as it helps to reduce overfitting, does not require normalization of data and works well with categorical (e.g. hero classes) and continuous variables (e.g. "in-game" statistics and metrics such as gold, experience, levels, etc.).

## 3. Data Cleaning
* * *
[Data Cleaning Report](https://github.com/robles2/Dota2-Analytics-Project/blob/master/Step%202%20-%20Data%20Wrangling.ipynb)

[Exploratory Data Analysis Report](https://github.com/robles2/Dota2-Analytics-Project/blob/master/Step%203%20-%20%20Exploratory%20Data%20Analysis.ipynb)

[Pre-processing Report](https://github.com/robles2/Dota2-Analytics-Project/blob/master/Step%204%20-%20Pre-Processing.ipynb)

The Data Cleaning took place progressively accross various steps of the project (Data Wrangling + EDA + Pre-processing). There are 3 main reasons for this:

1. In **Step 2 - Data Wrangling** there were too many datasets and it was necessary to understand their content to either select or discard them. In this step many files were discarded, and also many columns within the files selected were dropped due to containing information that was not useful for the project. Mainly, the work peformed here was selecting useful datasets and variables within those datasets and discarding others.


2. In the **Step 3 - Exploratory Data Analysis**, missing values were imputed in the datasets previously selected and different calculations and aggregations were made. As a result 2 datasets were generated and saved in this step, which are:

- **Categories data**: For every match available, shows the 10 players, what heroe each one of them picked and what are the categories of such heroes (we created in the EDA a total of 3 categories "hero function", "hero farm" and "hero powerspike".

  - **Problem 1**: How to make clusters of heroes when every player plays each hero in a different way? In order to make clusters of heroes, I decided to first summarize the typical hero "behaviour" assuming that the median of the player's end game statistics was a good proxy to do the clustering. As a result, I obtained a total of 112 rows, one per hero, and several columns, each of which showed the median values of how players performed with that hero. In simpler terms, for each hero, how many times they killed, were killed, how much gold and experience they earned, how much damage they dealt, etc.
  
  - **Problem 2**: Some of the heroes behaviours were not easy to interpret based on the end game metrics. To understand the functions of the heroes, for example, we calculated in percentual terms, how much damage they dealt to either structures of enemy heroes, to better understand if the were more "pushers" (better at destroying structures) or "killers" (better at killing enemy heroes). We did the same for the sources of gold earned, to understand if they earned more percentage of their gold from killing neutral creeps or enemy heroes, etc. These variables were later on used as clustering variables.
  

 - **Time Data**: "In-game" minute by minute information at team basis, of gold, experience, gold per minute, experience per minute and the difference (advantage/disadvantage) that each team had on these variables per minute.
 
   - **Problem 1**: Information was not at team basis, but at player basis. We had to calculate the sum of each team member gold and experience, in order to calculate the total gold and experience earned, as well as the gold and experience per minute.

3. In **Step 4 - Pre-processing**, we had to transform both datasets to horizontal column based, binary-type information, so that there was one row per match, and as many columns as variables.

- **Model 1 - Prediction at minute 0**: In this model, we had to prepare the data so that all heores selected and their categories, without any "in-game" information. We had to get dummies for every team heroes picked as well as a count of how many categories of heroes existed on each team.

  - **Problem 1**: Data of categories had to be counted by each category and team. To do this we proceeded to transform the table to get this counts before adding them to the one row per match dataframe.
  
- **Model 2 - Prediction at minute 15**: In this model, we excluded the heroes picked by each team, but maintained the count of heroes categories. I added the gap of experience and gold per minute earned by each team at minute marks of 5, 8, 10, 12 and 15.

  - **Problem 1**: Information about minutes was in rows, we had to transform the table to include columns that would represent the minute marks of gold per minute and experience per minute earned at minutes 5, 8, 10, 12 and 15.

## 4. EDA
* * *
[Exploratory Data Analysis Report](https://github.com/robles2/Dota2-Analytics-Project/blob/master/Step%203%20-%20%20Exploratory%20Data%20Analysis.ipynb)

In the Exploratory Data Analysis we focused on answering two questions:
1. **Is there a way to group the different heroes into categories?**

To answer this question, firstly we focused on finding a good way to represent how the average player uses each hero. This is relevant to the extent that players use the same hero in very different ways. For example, one player may use a hero mainly to kill enemy heroes because they like that type of playstyle, and other may use the same hero in a defensive way, to protect allies. Consiering this, what is that hero's actual "role" or "category"?

We set as criteria for this actual "role" how the median player uses a hero, considering the end game median values for kills made, times died, assists made, damage dealt, gold earned, etc. We prefered the median over the average because it is less sensitive to distortions of outliers.

Secondly, once that we had identified the most representative hero usage, we calculated and extracted several variables per hero and proceded to make clusters based on these categories. In particular, we clustered the heroes in the following ways:

  - By hero functions: Into 6 categories that provided information about what behaviour is expected from that hero.
  - By hero farm dependency: Into 3 categories that provided information about how many neutral creeps that hero usually kills.
  - By hero powerspike: Into 3 categories taht provided information about how early or late the hero was most impactful in the game.
  
As a result we generated this map of heroes:

![alt text](classification.png)

2. **What metric/s exist that are useful in predicting what team will win the match?**

In relation with this analysis we split the players between winners and losers and proceeded to analyse their end game statistics to see if we found any big differences between their distributions. It was clear that winners had earned more gold and experience than losers. This could be observed regardless of the game duration. In particular the "gap" of difference between teams gold per minute and experience per minute through time seemed to be a very good indicator of what team was winning.

![alt text](Models2.PNG)


## 5. Modeling
* * *
[Pre-processing Report](https://github.com/robles2/Dota2-Analytics-Project/blob/master/Step%204%20-%20Pre-Processing.ipynb)

In this step, we focused on building 2 models, each one of them aiming to predict which team will win but at different moments in time. As baseline case, we calculated the probability of guessing right the winning team by always choosing the same team as winner. The result is that anybody would expect to guess right 51% of games if always team radiant was predicted as winner.

* The first model would predict which team would win at minute 0 of the game, only considering information available before the game started. We defined a Random Forest Classifier and fit the data, our model had a performance after crossvalidation and hyperparameter tunning of around 58%, this is an increase of 7% over the baseline case.


* The second model would predict which team would win at minute 15 of the match (average match lasts around 45 minutes), considering the "gap" of gold per minute and advantage per minute of each team at minute marks 5, 8, 10, 12 and 15. The Random Forest Classifier, after crossvalidation and hyperparameter tunning had a performance of around 70%, this is an incrase of 19% over the baseline case.

![alt text](performance2.PNG)

## 6. Future Improvements
* * *
This project has been great for starting to understand the Dota 2 game, but there are still many questions unsolved that would be very interesting to solve such as:

* Can we predict what will be the next team's hero to be selected in the draft?
* Can we recommend best items to buy for a hero/team?
* How does hero usage change over different skill levels?
* Can we recommend best abilities to upgrade for a hero?
* How can we estimate the skill level of a player?
* Can we estimate expected gpm and xpm advantage at certain minute?
* Can we estimate the game durations based on a combination of the drafts and gap in gpm/xpm?

We will try to work in the future on some of this questions to add new functionalities and imrpove our understanding of the game, aiming at developing a environment in which players and coaches can better understand team compositions and define and implement strategies.